# Data Preprocessing

In [24]:
# Load required packages
import pandas as pd
import numpy as np

### Load Data Set

This data set has been downloaded from the Convention on International Trade in Endangered Species of Wild Fauna and Flora (CITES) website, available through https://trade.cites.org/, on 31 May 2024. 

In [26]:
# Import original data set
CITES_full = pd.read_csv("Datasets/CITES31May.csv", sep = ";")

# Check dataset 
CITES_full.head(3)

,Year,App.,Taxon,Class,Order,Family,Genus,Importer,Exporter,Origin,Importer reported quantity,Exporter reported quantity,Term,Unit,Purpose,Source
0,1977,II,Manis javanica,Mammalia,Pholidota,Manidae,Manis,FR,GB,XX,NaN,4.0,skins,NaN,NaN,NaN
1,1977,II,Manis javanica,Mammalia,Pholidota,Manidae,Manis,US,ES,XX,215.0,NaN,shoes,NaN,NaN,NaN
2,1977,II,Manis javanica,Mammalia,Pholidota,Manidae,Manis,US,FR,ID,NaN,444.0,skins,NaN,NaN,NaN


Check that the DataFrame has been loaded correctly, i.e. with 16 named columns from `Year` to `Source`.

### Remove Unnecessary Variables

Several columns in the original data set will not be of use for this project, so we remove them now.

In [27]:
# Remove unnecessary variables from CITES data
CITES = CITES_full.drop(['App.', 'Class', 'Order', 'Family', 'Genus', 'Purpose', 'Source'], axis = 1)

### Select African and Asian Exporters

Pangolins only occur naturally in Africa and certain parts of Asia (Rotich, 2018), and are incredibly difficult to breed in captivity (Environmental Investigation Agency, 2024).  We make the assumption that all `Exporter` countries that are not in regions where pangolins occur naturally have imported their stock from elsewhere, and hence we remove these transactions to avoid double-counting the items traded.  See Appendix A - Country Codes for a list of country codes and their regions.

In [28]:
# Create lists of African and Asian countries
African_countries = ["ZA", "TG", "CG", "CF", "CM", "LR", "NG", "TZ", "BJ", "DJ", "CI", "GN", "GQ", 
                     "GA", "CD", "KE", "GH", "UG", "BI", "BW", "SZ", "SN", "MZ"]
Asian_countries = ["CN", "JP", "TH", "SG", "ID", "MY", "TW", "PH", "LA", "HK", "KR", "MO", "VN", 
                   "KH", "PK", "LK"]

In [29]:
# Remove entries for non-African and non-Asian countries, but keep those with no recorded exporter
CITES = CITES[CITES['Exporter'].isin(African_countries + Asian_countries + ['XX', np.nan])]

If you are using this code for a data set more recent than the one provided for this project, you may need to update the lists of African and Asian countries based on what appears in the `Exporter` column.  Refer to the article by V. Rotich for more information on which countries contain wild pangolin populations.

### Remove Double-Counted Transactions

According to the CITES Database Guidelines for Use (CITES Secretariat and UNEP-WCMC, 2022), there may be cases where one transaction is recorded twice in the data set - once using the importer's report and once using the exporter's report.  We must remove these instances to avoid counting the same transactions twice.

In [30]:
# Create index for which rows to remove
repeat_index = []

# Loop to find repeated transactions
for i in range(len(CITES)-1):
    # Extract required elements from the data set
    importer_now = CITES['Importer'].iloc[i]
    importer_next = CITES['Importer'].iloc[i+1]
    exporter_now = CITES['Exporter'].iloc[i]
    exporter_next = CITES['Exporter'].iloc[i+1]
    importer_quantity_now = CITES['Importer reported quantity'].iloc[i]
    importer_quantity_next = CITES['Importer reported quantity'].iloc[i+1]
    exporter_quantity_now = CITES['Exporter reported quantity'].iloc[i]
    exporter_quantity_next = CITES['Exporter reported quantity'].iloc[i+1]
    
    # Check for repeated importer and exporter
    if (importer_now == importer_next) & (exporter_now == exporter_next ):
        # Check for repeated quantities
        if (importer_quantity_now == exporter_quantity_next) | (exporter_quantity_now == importer_quantity_next):
            repeat_index.append(i)

    # Check for resales
    if (exporter_now == importer_next) & (CITES['Origin'].iloc[i] == exporter_next):
        if (importer_quantity_now == exporter_quantity_next) | (exporter_quantity_now == importer_quantity_next):
            repeat_index.append(i)

I further inspected the data and found a few anomalies.  There is no way to code a function to find these so I will remove them manually.

In [31]:
# Add the manual indices to repeat_index
repeat_index = repeat_index + [379, 412, 451, 485, 487, 493, 593, 663, 1014, 1029, 1073, 1095, 1130]
repeat_index = sorted(repeat_index)

In [32]:
# Remove the selected rows from the dataframe
CITES.drop(CITES.index[repeat_index], inplace=True)

Finally, we combine importer and exporter quantities to get one `Quantity` value per transaction.  According to the CITES Guidelines (CITES Secretariat and UNEP-WCMC, 2022), the exporter reported quantity is generally more reliable, so we will use that value where it is available.  Otherwise, we will use the importer reported quantity.

In [33]:
# Copy Exporter reported quantity for Quantity
CITES['Quantity'] = CITES['Exporter reported quantity']

# Find column number of the 'Quantity' column
quan_ind = CITES.columns.get_loc('Quantity')

# Fill in NaN Quantity values with Importer reported quantity
for i in range(len(CITES)):
    if np.isnan(CITES.iloc[i,quan_ind]):
        CITES.iloc[i,quan_ind] = CITES['Importer reported quantity'].iloc[i]

In [34]:
# Remove the Exporter and Importer reported quantity columns
CITES = CITES.drop(['Exporter reported quantity', 'Importer reported quantity'], axis = 1)

### Indicator for African or Asian Species

Some entries in the data set list the `Taxon` as "Manidae spp." or "Manis spp.".  These are general terms, so in this case we will assign the pangolins to the region of their `Origin` if it is listed, and if not then we assign it to the `Exporter`'s region.

In [35]:
# Lists of species per region
African_species = ["Manis tetradactyla", "Manis tricuspis", "Manis gigantea", "Manis temminckii"]
Asian_species = ["Manis crassicaudata", "Manis javanica", "Manis culionensis", "Manis pentadactyla"]

In [36]:
# Create empty variable for Region
CITES['Region'] = ['Other']*len(CITES)
reg_ind = CITES.columns.get_loc('Region')

# Loop through to assign Region
for i in range(len(CITES)):
    if CITES['Taxon'].iloc[i] in African_species: 
        CITES.iloc[i,reg_ind] = "African"
    elif CITES['Taxon'].iloc[i] in Asian_species: 
        CITES.iloc[i,reg_ind] = "Asian"
    elif CITES['Origin'].iloc[i] in African_countries: 
        CITES.iloc[i,reg_ind] = "African"
    elif CITES['Origin'].iloc[i] in Asian_countries: 
        CITES.iloc[i,reg_ind] = "Asian"
    elif CITES['Exporter'].iloc[i] in African_countries: 
        CITES.iloc[i,reg_ind] = "African"
    elif CITES['Exporter'].iloc[i] in Asian_countries: 
        CITES.iloc[i,reg_ind] = "Asian"

# Convert to factor
CITES['Region'] = CITES['Region'].astype('category')

### Calculate Number of Pangolins per Transaction

Each row in the data set lists a `Term`, detailing the type of product sold (e.g. skins, meat, specimens) and a `Unit` (e.g. kg, number of specimens, cartons).  We now convert these to the equivalent number of pangolins required to produce each `Term`.  See Appendix B - Term Classification and Conversion for more details.

In [38]:
# Set up a list for all the terms where 1 unit equates to 1 pangolin
direct_equiv = ["bodies", "carvings", "claws", "leather", "leather items", "leather products (large)", 
                "live", "shoes", "skeletons", "skulls", "tails", "trophies"]

# Create data frame with the average weight for each species of pangolin
weights = {'Species': ["Manis gigantea", "Manis crassicaudata", "Manis temminckii", "Manis javanica", 
                       "Manis pentadactyla", "Manis tetradactyla", "Manis culionensis", 
                       "Manis tricuspis", "Manis spp.", "Manidae spp."], 
           'Weight kg': [33, 13, 12, 4.9, 3.6, 2.7, 2.1, 1.5, 9.1, 9.1]}
weights_df = pd.DataFrame(weights)

In [39]:
# Create column in CITES for number of pangolins
CITES['Number'] = [0.0]*len(CITES)
num_ind = CITES.columns.get_loc('Number')

# Create column in CITES for kg (to be removed once we have populated the Number column)
CITES['kg'] = [0.0]*len(CITES)
kg_ind = CITES.columns.get_loc('kg')

# Fill in kg values for each entry
for i in range(len(CITES)):
    for j in range(len(weights_df)):
        if CITES['Taxon'].iloc[i] == weights_df['Species'].iloc[j]:
            CITES.iloc[i,kg_ind] = CITES['Quantity'].iloc[i]/weights_df['Weight kg'].iloc[j]

Due to the wide variety of combinations in Term and Unit, we have a slightly complicated process for converting these to the overall number of whole pangolins traded.  This is detailed below.

In [40]:
# Find Number for each entry
for i in range(len(CITES)):
    # Extract relevant values from the dataset
    i_term = CITES['Term'].iloc[i]
    i_unit = CITES['Unit'].iloc[i]
    i_quantity = CITES['Quantity'].iloc[i]
    i_kg = CITES['kg'].iloc[i]

    # Check for directly equivalent quantities, by term or unit or both
    if (i_term in direct_equiv) | (i_unit in ['Number of specimens', 'pairs', 'flasks']) | (pd.isnull(i_unit) & 
            (i_term in ['medicine', 'skins', 'specimens', 'unspecified'])): 
        CITES.iloc[i,num_ind] = i_quantity    

    # For units measured in kg, g, and ml according to total bodyweight of pangolin
    elif i_term in ['derivatives', 'medicine', 'powder', 'specimens', 'unspecified']:  
        if (i_unit == 'kg') | (pd.isnull(i_unit)):
            CITES.iloc[i,num_ind] = i_kg
        elif i_unit in ['g', 'ml']:
            CITES.iloc[i,num_ind] = i_kg/1000
        elif i_unit in ['boxes', 'cartons']:
            CITES.iloc[i,num_ind] = 20*i_kg

    # For 'scales', 'skin pieces', and 'skins'
    elif i_term in ['scales', 'skin pieces', 'skins']:
        if (i_unit == 'kg') | pd.isnull(i_unit): 
            CITES.iloc[i,num_ind] = i_kg/0.2
        elif i_unit == 'g':
            CITES.iloc[i,num_ind] = i_kg/200
        elif i_unit == 'cartons':
            CITES.iloc[i,num_ind] = 100*i_kg
        elif i_unit in ['m', 'm2']:
            CITES.iloc[i,num_ind] = i_quantity/1.2
        elif i_unit == 'cm':
            CITES.iloc[i,num_ind] = i_quantity/120

    # For 'meat'
    elif i_term == 'meat':
        if i_unit == 'g': CITES.iloc[i,num_ind] = i_kg/300
        else: CITES.iloc[i,num_ind] = i_kg/0.3

    # For all 'feet'
    elif i_term == 'feet': 
        CITES.iloc[i,num_ind] = i_quantity/4
    
    # For all 'leather products (small)' and 'fur product (small)'
    elif i_term in ['leather products (small)', 'fur product (small)']: 
        CITES.iloc[i,num_ind] = i_quantity/2

Now that we have all the conversions, we make one final assumption, which is that one cannot kill a fraction of a pangolin. We therefore will round all `Number` values up to the nearest whole number.  We remove the 'kg' variable as it is no longer needed.  

In [41]:
# Round up all Number values
CITES['Number'] = np.ceil(CITES['Number'])

# Remove 'kg' column
CITES = CITES.drop(['kg'], axis = 1) 

### Grouping of Trade Purposes

There are many different types of `Term` items listed, so I would like to group similar ones together to simplify the analysis.  For example, "leather" and "leather items" could be joined together into one group.

In [42]:
# Create lists for groupings
groupnames = ["whole", "trophy", "skins", "consumption", "derivatives", "medicine"]
whole = ["bodies", "live", "specimens"]
trophy = ["trophies", "carvings", "shoes"]
skins = ["fur product (small)", "leather", "leather items", "leather products (large)", 
         "leather products (small)", "scales", "skin pieces", "skins"]
consumption = ["meat"]
derivatives = ["derivatives", "feet", "claws", "skeletons", "skulls", "tails", "unspecified"]
medicine = ["medicine", "powder"]
all_groups = [whole, trophy, skins, consumption, derivatives, medicine]

In [43]:
# Create new column in CITES to store the group name
CITES['Purpose'] = ['h']*len(CITES)
pur_ind = CITES.columns.get_loc('Purpose')

In [44]:
# Loop through all Terms to group them according to purpose
for i in range(len(CITES)):
    for j in range(len(groupnames)):
        if CITES['Term'].iloc[i] in all_groups[j]:
            CITES.iloc[i, pur_ind] = groupnames[j]

In [45]:
# Reset index
CITES = CITES.reset_index(drop = True)

In [46]:
# Final check
CITES.head()

,Year,Taxon,Importer,Exporter,Origin,Term,Unit,Quantity,Region,Number,Purpose
0,1977,Manis pentadactyla,AU,CN,NaN,live,NaN,2.0,Asian,2.0,whole
1,1978,Manis crassicaudata,US,JP,XX,skins,NaN,298.0,Asian,298.0,skins
2,1978,Manis javanica,US,JP,XX,skins,NaN,1016.0,Asian,1016.0,skins
3,1978,Manis javanica,US,TH,NaN,live,NaN,1.0,Asian,1.0,whole
4,1978,Manis pentadactyla,AU,SG,NaN,leather products (small),NaN,1.0,Asian,1.0,skins


Finally we have our 'tidied' data set, which contains columns indicating the type, unit, and quantity of pangolin products traded; the importer, exporter, and origin; the species and region of origin of each pangolin; the number of whole pangolins involved per trade; and a broad grouping of different purposes for pangolin products.  We are now ready to export this data set and begin our analysis.

In [47]:
# Export the tidied data set
CITES.to_csv('Datasets/CITES.csv')

### References

1. CITES Secretariat and UNEP-WCMC (2022). A guide to using the CITES Trade Database. 
Version 9. Geneva, Switzerland, and Cambridge, UK. https://trade.cites.org/cites_trade_guidelines/en-CITES_Trade_Database_Guide.pdf
2. Environmental Investigation Agency. (2024). Saving Pangolins from Extinction. Retrieved from Environmental Investigation Agency: https://eia-international.org/wildlife/helping-pangolins/saving-pangolins-from-extinction/
3. Rotich, V. (2018, September 04). Where Do Pangolins Live. Retrieved from WorldAtlas: https://www.worldatlas.com/articles/what-is-a-pangolin.html